<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import seaborn as sb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn import preprocessing
import timeit

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [2]:
# # Load data
df_train = pd.read_parquet('../data/training_set.parquet', engine = 'auto')
#df_train = df_train[df_train['srch_id'] < 10000]
df_test = pd.read_parquet('../data/test_set.parquet', engine = 'auto')

<h1>Data prep<h1\>

In [3]:
def make_score(df):
    df.loc[:, 'score'] = np.zeros(len(df))
    df.loc[df['click_bool'] == 1, 'score'] = 1
    df.loc[df['booking_bool'] == 1, 'score'] = 5
    return df

# Add features for hour, day and month.
def date_time(df):
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['hour'] = df['date_time'].dt.hour
    df['day'] = df['date_time'].dt.day
    df['month'] = df['date_time'].dt.month
    df = df.drop('date_time', axis=1)
    return df

def remove_cols(df, cols):
    return df.drop(cols, axis=1)

def remove_cols_nan(df, limit):
    for col in df.columns:
        if len(df[col]) * limit < df[col].isna().sum():
            df = df.drop(col, axis=1)
    return df

# Add column with a ranking for each property in a search based on another column.
def create_rank_feature(df, col):
    df['rank_' + str(col)] = df.groupby('srch_id')[col].rank(ascending=False)
    return df

# location score 2 has missing values for property on some rows while some rows have a score.
# take average of rows that do have a score. reduces nans from 1090348 to 182213.
def fill_location_score_2(df):
    df['prop_location_score2'] = df.groupby('prop_id')['prop_location_score2'].transform(lambda x: x.fillna(x.mean()))
    return df

# adds a normalised version of a column based on a chosen grouping.
def add_normalized_column(df, col, group):
    df['norm_' + str(col) + "_" + str(group)] = (
        (df[col] - df.groupby(group)[col].transform('mean')) 
        / df.groupby(group)[col].transform('std')
    )
    return df

def prep_data(df, target_cols, test=False):
    df_new = df.copy()
    
    if not test:
        print('add score')
        df_new = make_score(df_new)
        df_new = df_new.drop(target_cols, axis=1)
  

    print('add hour, day, month')
    df_new = date_time(df_new)
    #df_new = remove_cols_nan(df_new, 0.9)
    
    # difference features assumes that users purchase in same category as history.
    print('add difference features')
    df_new['usd_diff'] = df_new['visitor_hist_adr_usd'] - df_new['price_usd']
    df_new['star_diff'] = df_new['visitor_hist_starrating'] - df_new['prop_starrating']    
    df_new['log_price_diff'] = df_new['prop_log_historical_price'] - np.log(df_new['price_usd'])
    
    # count variables 
    # theory: A property that is in more searches is purchased more often.
    df_new['prop_id_count'] = df.groupby('prop_id')['prop_id'].transform('count')
    
    df_new['srch_destination_id_count'] = df.groupby('srch_destination_id')['srch_destination_id'].transform('count')

    
    # ranking features
    print('add rank features')
    df_new = create_rank_feature(df_new, 'price_usd')
    df_new = create_rank_feature(df_new, 'prop_starrating')
    df_new = create_rank_feature(df_new, 'prop_review_score')
    df_new = create_rank_feature(df_new, 'prop_location_score1')
    df_new = create_rank_feature(df_new, 'prop_location_score2')
    
    print("fill ls2")
    df_new = fill_location_score_2(df_new)
    
    # Fill distance with mean.
    df['orig_destination_distance'].fillna(df['orig_destination_distance'].mean(),inplace=True)
    
    
    print("remove nan")
    # Fill rest of nan values with lowest.
    for i in df_new.columns[df_new.isnull().any(axis=0)]:
        df_new[i].fillna(0,inplace=True)
    
    print('add normalised features')
    groups = ['srch_id', 'prop_country_id', 'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window', 'month']
    targets = ['price_usd', 'prop_starrating', 'prop_review_score', 'prop_location_score1', 'prop_location_score2']
    for group in groups:
        for target in targets:
            df_new = add_normalized_column(df_new, target, group)
   
    df_new = add_normalized_column(df_new, 'price_usd', 'prop_id')
    df_new = add_normalized_column(df_new, 'price_usd', 'srch_room_count')
    
    # Normalisation might create nans
    print("remove nan\n")
    for i in df_new.columns[df_new.isnull().any(axis=0)]:
        df_new[i].fillna(0,inplace=True)
        
    for c in categorical_features:
        df_new[c] = df_new[c].astype('category')
    
    return df_new

In [4]:
target_cols = ['click_bool', 'booking_bool', 'gross_bookings_usd', 'position']
categorical_features = ['hour', 'day', 'month', 'site_id', 'visitor_location_country_id', 'prop_country_id', 'prop_id', 'srch_destination_id']

print("Prepping test data")
df_test = prep_data(df_test, target_cols, True)
print("Prepping training data")
df_train = prep_data(df_train, target_cols, False)


Prepping test data
add hour, day, month
add difference features


/Users/hrvanelderen/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


add rank features
fill ls2
remove nan
add normalised features
remove nan

Prepping training data
add score
add hour, day, month
add difference features


/Users/hrvanelderen/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


add rank features
fill ls2
remove nan
add normalised features
remove nan



In [5]:
len(df_train.columns)

95

<h1>Data split<h1\>

In [6]:
from sklearn.model_selection import GroupShuffleSplit

splitter = GroupShuffleSplit(test_size=0.2, n_splits=2, random_state = 7)
split = splitter.split(df_train, groups=df_train['srch_id'])
train_inds, test_inds = next(split)

df_ideal = df_train.iloc[test_inds].copy().sort_values(by=['srch_id', 'score'], ascending=[True, False], inplace=False)

X = df_train.drop(['score'], axis=1)
y = df_train['score']

X_train, X_test, y_train, y_test, test_ideal = X.iloc[train_inds], X.iloc[test_inds], y.iloc[train_inds], y.iloc[test_inds], df_ideal

train_groups = X_train.groupby('srch_id').size().to_frame('size')['size'].to_numpy()

test_groups = X_test.groupby('srch_id').size().to_frame('size')['size'].to_numpy()


<h1>Training <h1\>

In [7]:
params = {
    "objective": "lambdarank",
    "boosting_type":"dart",
    "metric":"ndcg",
    'n_estimators': 500, 
    'max_depth': 10, 
    'learning_rate': 0.08925380432712779, 
    'subsample': 0.523890758165789, 
    'colsample_bytree': 0.5433556425106324, 
    'feature_fraction': 0.9677058301342538,
    'reg_alpha': 0.00011669441178850782, 
    'reg_lambda': 0.008250891056480582
}

gbm = lgb.LGBMRanker(**params)


In [8]:
gbm.fit(
    X=X_train,
    y=y_train,
    group=train_groups,
    eval_set=[(X_train, y_train),(X_test, y_test)],
    eval_group=[train_groups, test_groups],
    eval_at=[5],
    feature_name='auto', 
    categorical_feature = 'auto')


[LightGBM] [Warning] feature_fraction is set=0.9677058301342538, colsample_bytree=0.5433556425106324 will be ignored. Current value: feature_fraction=0.9677058301342538


/Users/hrvanelderen/anaconda3/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/hrvanelderen/anaconda3/lib/python3.10/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1]	training's ndcg@5: 0.314398	valid_1's ndcg@5: 0.308587
[2]	training's ndcg@5: 0.349608	valid_1's ndcg@5: 0.337742
[3]	training's ndcg@5: 0.362457	valid_1's ndcg@5: 0.348189
[4]	training's ndcg@5: 0.37095	valid_1's ndcg@5: 0.354145
[5]	training's ndcg@5: 0.377096	valid_1's ndcg@5: 0.35734
[6]	training's ndcg@5: 0.380535	valid_1's ndcg@5: 0.359582
[7]	training's ndcg@5: 0.384123	valid_1's ndcg@5: 0.361027
[8]	training's ndcg@5: 0.384617	valid_1's ndcg@5: 0.361647
[9]	training's ndcg@5: 0.388722	valid_1's ndcg@5: 0.364217
[10]	training's ndcg@5: 0.392096	valid_1's ndcg@5: 0.365759
[11]	training's ndcg@5: 0.39443	valid_1's ndcg@5: 0.367582
[12]	training's ndcg@5: 0.394529	valid_1's ndcg@5: 0.368003
[13]	training's ndcg@5: 0.395835	valid_1's ndcg@5: 0.368471
[14]	training's ndcg@5: 0.397747	valid_1's ndcg@5: 0.36981
[15]	training's ndcg@5: 0.399365	valid_1's ndcg@5: 0.370259
[16]	training's ndcg@5: 0.401667	valid_1's ndcg@5: 0.371299
[17]	training's ndcg@5: 0.403048	valid_1's ndcg@5: 0.

[138]	training's ndcg@5: 0.454974	valid_1's ndcg@5: 0.387388
[139]	training's ndcg@5: 0.455674	valid_1's ndcg@5: 0.387377
[140]	training's ndcg@5: 0.455641	valid_1's ndcg@5: 0.387262
[141]	training's ndcg@5: 0.455636	valid_1's ndcg@5: 0.387393
[142]	training's ndcg@5: 0.455537	valid_1's ndcg@5: 0.387324
[143]	training's ndcg@5: 0.455981	valid_1's ndcg@5: 0.387969
[144]	training's ndcg@5: 0.456064	valid_1's ndcg@5: 0.387996
[145]	training's ndcg@5: 0.456478	valid_1's ndcg@5: 0.388114
[146]	training's ndcg@5: 0.456379	valid_1's ndcg@5: 0.387785
[147]	training's ndcg@5: 0.456399	valid_1's ndcg@5: 0.38785
[148]	training's ndcg@5: 0.456422	valid_1's ndcg@5: 0.387982
[149]	training's ndcg@5: 0.456939	valid_1's ndcg@5: 0.388091
[150]	training's ndcg@5: 0.457441	valid_1's ndcg@5: 0.388168
[151]	training's ndcg@5: 0.457567	valid_1's ndcg@5: 0.388135
[152]	training's ndcg@5: 0.457631	valid_1's ndcg@5: 0.388316
[153]	training's ndcg@5: 0.458019	valid_1's ndcg@5: 0.387932
[154]	training's ndcg@5: 

[273]	training's ndcg@5: 0.483916	valid_1's ndcg@5: 0.392874
[274]	training's ndcg@5: 0.483813	valid_1's ndcg@5: 0.392864
[275]	training's ndcg@5: 0.484357	valid_1's ndcg@5: 0.39266
[276]	training's ndcg@5: 0.484348	valid_1's ndcg@5: 0.392586
[277]	training's ndcg@5: 0.484469	valid_1's ndcg@5: 0.392588
[278]	training's ndcg@5: 0.484397	valid_1's ndcg@5: 0.39251
[279]	training's ndcg@5: 0.484762	valid_1's ndcg@5: 0.392853
[280]	training's ndcg@5: 0.484777	valid_1's ndcg@5: 0.392837
[281]	training's ndcg@5: 0.48483	valid_1's ndcg@5: 0.392892
[282]	training's ndcg@5: 0.48529	valid_1's ndcg@5: 0.392944
[283]	training's ndcg@5: 0.485273	valid_1's ndcg@5: 0.39298
[284]	training's ndcg@5: 0.485368	valid_1's ndcg@5: 0.392917
[285]	training's ndcg@5: 0.485884	valid_1's ndcg@5: 0.392942
[286]	training's ndcg@5: 0.48644	valid_1's ndcg@5: 0.39319
[287]	training's ndcg@5: 0.486853	valid_1's ndcg@5: 0.393239
[288]	training's ndcg@5: 0.486818	valid_1's ndcg@5: 0.393208
[289]	training's ndcg@5: 0.4875

[408]	training's ndcg@5: 0.507337	valid_1's ndcg@5: 0.39506
[409]	training's ndcg@5: 0.507568	valid_1's ndcg@5: 0.395064
[410]	training's ndcg@5: 0.507961	valid_1's ndcg@5: 0.39537
[411]	training's ndcg@5: 0.508493	valid_1's ndcg@5: 0.395474
[412]	training's ndcg@5: 0.508415	valid_1's ndcg@5: 0.395339
[413]	training's ndcg@5: 0.508318	valid_1's ndcg@5: 0.395322
[414]	training's ndcg@5: 0.508624	valid_1's ndcg@5: 0.395193
[415]	training's ndcg@5: 0.508942	valid_1's ndcg@5: 0.395014
[416]	training's ndcg@5: 0.50942	valid_1's ndcg@5: 0.394988
[417]	training's ndcg@5: 0.509355	valid_1's ndcg@5: 0.395003
[418]	training's ndcg@5: 0.509322	valid_1's ndcg@5: 0.395079
[419]	training's ndcg@5: 0.509652	valid_1's ndcg@5: 0.394979
[420]	training's ndcg@5: 0.509633	valid_1's ndcg@5: 0.394894
[421]	training's ndcg@5: 0.509833	valid_1's ndcg@5: 0.394879
[422]	training's ndcg@5: 0.510196	valid_1's ndcg@5: 0.395038
[423]	training's ndcg@5: 0.510474	valid_1's ndcg@5: 0.394968
[424]	training's ndcg@5: 0.

LGBMRanker(boosting_type='dart', colsample_bytree=0.5433556425106324,
           feature_fraction=0.9677058301342538,
           learning_rate=0.08925380432712779, max_depth=10, metric='ndcg',
           n_estimators=500, objective='lambdarank',
           reg_alpha=0.00011669441178850782, reg_lambda=0.008250891056480582,
           subsample=0.523890758165789)

In [9]:
# # Optimize LGBM with optuna
# import optuna
# from functools import partial

# def objective(trial, X_train, y_train, X_test, test_ideal):
#     #y_train_gbm = y_train.astype(int)
#     #y_train_gbm[y_train == 5] = 2

#     params = {
#         "objective": "lambdarank",
#         "metric":"ndcg",
#         "random_state": 42,
#         "boosting_type": "dart",
#         #"early_stopping_round": trial.suggest_int("early_stopping_round", 10, 50),
#         "n_estimators": trial.suggest_int("n_estimators", 100, 500),
#         "num_leaves": trial.suggest_int("num_leaves", 20, 200),
#         "max_depth": trial.suggest_int("max_depth", 3, 10),
#         "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
#         "subsample": trial.suggest_float("subsample", 0.2, 1.0),
#         "reg_alpha": trial.suggest_float("reg_alpha", 1e-4, 1e-1, log=True),
#         "reg_lambda": trial.suggest_float("reg_lambda", 1e-4, 1e-1, log=True),
#         "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
#     }

#     gbm = lgb.LGBMRanker(**params)
#     gbm.fit(X_train, y_train, group=train_groups, eval_set=[(X_test, y_test)], eval_group=[test_groups])

#     pred_lgbm = constructs_predictions(gbm, X_test, ideal_df=test_ideal)
#     ndcg = calc_NDCG(test_ideal, pred_lgbm)

#     return ndcg

# print("Training LGBM")

# # Wrap the objective function with the input data
# objective_with_data = partial(objective, X_train=X_train, y_train=y_train, X_test=X_test, test_ideal=test_ideal)

# # Create an Optuna study and optimize the objective function
# study = optuna.create_study(direction="maximize")
# study.optimize(objective_with_data, n_trials=20)



In [10]:
# best_params = study.best_params
# print(best_params)

In [11]:
# best_params = study.best_params
# gbm = lgb.LGBMRanker(objective="lambdarank", metric="ndcg", boosting_type="dart", **best_params)
# gbm.fit(X_train, y_train, group=train_groups, eval_set=[(X_test, y_test)], eval_group=[test_groups])

<h1> Validation <h1\>

In [12]:
def construct_pred_ideal(df_in, df_ideal, y_pred):
    df = df_in.copy()
    df['pred_grades'] = y_pred
    df = df.sort_values(by=['srch_id', 'pred_grades'], ascending=[True, False], inplace=False)

    # Merge grades from ideal on srch_id and prop_id
    df = df.merge(df_ideal, on=['srch_id', 'prop_id'], how='left')

    # Return srch_id, prop_id and pred_grades
    return df[['srch_id', 'prop_id', 'pred_grades', 'score']]

def construct_pred_submission(df_in, y_pred):
    df = df_in.copy()
    df['pred_grades'] = y_pred
    df = df.sort_values(by=['srch_id', 'pred_grades'], ascending=[True, False], inplace=False)

    # Return srch_id, prop_id and pred_grades
    return df[['srch_id', 'prop_id']]

def constructs_predictions(model, data, ideal_df = None):
    y_pred = model.predict(data)

    if ideal_df is not None:
        pred_df = construct_pred_ideal(data, test_ideal, y_pred)
    else:
        pred_df = construct_pred_submission(data, y_pred)
    return pred_df

def calc_NDCG(df_ideal, df_pred, k = 5):
    # Group by 5
    df_ideal = df_ideal.groupby('srch_id').head(k)
    df_pred = df_pred.groupby('srch_id').head(k)

    assert df_ideal.shape[0] % k == 0
    assert df_pred.shape[0] % k == 0
    
    # Get grades matrices
    ideal_grades = df_ideal['score'].values.reshape(int(df_ideal.shape[0] / k), k)
    pred_grades = df_pred['score'].values.reshape(int(df_pred.shape[0] / k), k)

    discount_vec = [1/np.log2(i+2) for i in range(k)]

    # Calculate NDCG
    NDCG = (pred_grades @ discount_vec).sum() / (ideal_grades @ discount_vec).sum()

    return NDCG

In [ ]:
pred_lgbm = constructs_predictions(gbm, X_test, ideal_df=test_ideal)
pred_random = construct_pred_ideal(X_test, test_ideal, np.random.rand(len(X_test)))
pred_lgbm_submission = constructs_predictions(gbm, df_test)

Highest score: 0.4167321465628548

In [ ]:
print(f"LGBM: {calc_NDCG(test_ideal, pred_lgbm)},\nRandom: {calc_NDCG(test_ideal, pred_random)}")

In [ ]:
pred_lgbm_submission.to_csv('../data/submission_LGBM.csv', index=False)

In [ ]:
lgb.plot_importance(gbm, figsize = (12,20))